In [2]:
import sys
sys.path.append('../')

import numpy as np

from ForwardModeling.ForwardProcessing1D import forward_with_trace_calcing
from Inversion.Strategies.SeismDiffInversion1D import inverse
from Inversion.Optimizators.Optimizations import LBFGSBOptimization, DifferentialEvolution, TrustKrylov, TrustConstr, \
    ConjugateGradient
from Tests.test_ForwardProcessing1D import get_model_2layered, get_model_3layered
from Data.geol_models import get_model_3layered_fluid_rp_dicts
from Objects.Data.WavePlaceholder import OWT, WaveDataPlaceholder
from Objects.Models.Models import SeismicModel1D
from Objects.Attributes.RockPhysics.RockPhysicsAttribute import RockPhysicsAttribute
from Objects.Attributes.Seismic.SeismicAttribute import SeismicAttribute
from Objects.Models.Layer1D import Layer1D, LayerOPT
import time

from matplotlib import pyplot as plt
from Inversion.Strategies.SeismDiffInversion1D import func_to_optimize


In [3]:
def plot_1D_err(forward_params, nvals, min_max=None, dot=False, vline_x=None):
    
    if min_max is None:
        min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
        max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
        
    else:
        min_val = np.array([min_max[0]])
        max_val = np.array([min_max[1]])
    
    dval = (max_val - min_val) / nvals

    val_x = [min_val + dval*i for i in range(nvals+1)]
    errs = []
    for val in val_x:
        errs.append(func_to_optimize(val, placeholders, forward_params, helper=None, show_tol=False))
        
    plt.plot(val_x, errs)
    if dot:
        plt.plot(val_x, errs, 'o')
        
    if vline_x is not None:
        plt.vlines(vline_x, ymin=min(errs), ymax=max(errs), colors='r')
        
    plt.show()

def optimization_func_2D(forward_params, x, y):
    indexes_1 = np.arange(x.shape[0])
    indexes_2 = np.arange(x.shape[1])
    
    Z = np.zeros((x.shape[0], x.shape[1]))
    
    for ind1 in indexes_1:
        for ind2 in indexes_2:
            Z[ind1, ind2] = func_to_optimize([x[ind1, ind2], y[ind1, ind2]], placeholders, forward_params, helper=None, show_tol=False)
        
    return Z

def plot_2D_err(forward_params, nvals, ncontous=10, points=None):
    min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
    max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
    
    dval = (max_val - min_val) / nvals

    val_1_x = [min_val[0] + dval[0]*i for i in range(nvals[0]+1)]
    val_2_x = [min_val[1] + dval[1]*i for i in range(nvals[1]+1)]

    X, Y = np.meshgrid(val_1_x, val_2_x)

    Z = optimization_func_2D(forward_params, X, Y)
    
    plt.contourf(X, Y, Z, ncontous, cmap='seismic')
    plt.colorbar()
    
    if points is not None:
        plt.plot(points[0], points[1], marker='x', color='r')

# Четырехслойная модель с газоосадками

In [4]:
h = [
    67.5,
    22.5,
    20
]

In [5]:
layer_1_seism = {
    'vp': 1500,
    'vs': 0,
    'rho': 1000
}

layer_2_rp = {
    "name": "xu-payne",
    "components": {
                    "Km": {
                      "value": 7.3,
                      "optimize": False,
                      "min": 5,
                      "max": 10
                    },
					"Gm": {
                      "value": 2.71,
                      "optimize": False,
                      "min": 0.1,
                      "max": 5
                    },
					"rho_m": {
                      "value": 2.71,
                      "optimize": False,
                      "min": 2.5,
                      "max": 3
                    },
                    "Vm": {
                      "value": 1,
                      "optimize": False,
                      "min": 0.7,
                      "max": 1
                    },
                    "Ks": {
                      "value": 9.2,
                      "optimize": False,
                      "min": 7,
                      "max": 14
                    },
					"Gs": {
                      "value": 0.4,
                      "optimize": False,
                      "min": 0.001,
                      "max": 0.8
                    },
					"rho_s": {
                      "value": 2.43,
                      "optimize": False,
                      "min": 2,
                      "max": 3
                    },
                    "phi_s": {
                      "value": 0.19,
                      "optimize": False,
                      "min": 0.05,
                      "max": 0.3
                    },
                    "Kf": {
                      "value": 1.8,
                      "optimize": False,
                      "min": 0.5,
                      "max": 3
                    },
                    "rho_f": {
                      "value": 0.95,
                      "optimize": False,
                      "min": 0.01,
                      "max": 3
                    },
                    "phi": {
                      "value": 0.76,
                      "optimize": True,
                      "min": 0.5,
                      "max": 0.99
                    }
                  }
}

layer_3_seism = {
    'vp': 2000,
    'vs': 1000,
    'rho': 2070
}

# посмотреть в дипломе Машки!
layer_4_seism = {
    'vp': 6600,
    'vs': 3600,
    'rho': 2600,
}

In [7]:
layer_1 = Layer1D(h[0],
                  rp_attribute=None,
                  seism_attribute=SeismicAttribute(**layer_1_seism),
                  opt=LayerOPT.NO)
layer_2 = Layer1D(h[1],
                  rp_attribute=RockPhysicsAttribute(layer_2_rp['components'], layer_2_rp['name']),
                  seism_attribute=None,
                  opt=LayerOPT.RP)
layer_3 = Layer1D(h[2],
                  rp_attribute=None,
                  seism_attribute=SeismicAttribute(**layer_3_seism),
                  opt=LayerOPT.NO)
layer_4 = Layer1D(-1,
                  rp_attribute=None,
                  seism_attribute=SeismicAttribute(**layer_4_seism),
                  opt=LayerOPT.NO)

In [8]:
model = SeismicModel1D([layer_1, layer_2, layer_3, layer_4])

In [14]:
# Многоканальная коса

In [9]:
dx = 5
nx = 100
x_rec = [i * dx for i in range(1, nx+1)]
wave_types = [OWT.PdPu]

In [ ]:
observe, test_seismic = \
    forward_with_trace_calcing(model, x_rec,
                               dt=3e-03, trace_len=1500, wavetypes=wave_types, display_stat=True,
        visualize_res=False)

Calculating rockphysics model...
